In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import pickle
from tqdm import tqdm

# WMCA LA Code

In [2]:
# Get all local authority codes and corresponding council names
page = requests.get("https://epc.opendatacommunities.org/docs/api/domestic#domestic-local-authority").text
soup = BeautifulSoup(page)

table = soup.findAll('table')[3]
la_code_dict = {}

for tr in table.findAll('tr')[1:]:
   code, local_auth = tr.findAll('td')
   la_code_dict[local_auth.text] = code.text

# Get local authority codes for councils in WMCA
WMCA_councils = open("../data/raw/WMCA_council.txt").read().split(",")[:-1]
WMCA_code = [la_code_dict[i] for i in WMCA_councils]
WMCA = dict(zip(WMCA_code, WMCA_councils))

# Save codes for future use
with open('../data/raw/WMCA_council_code.pkl', 'wb') as f:
    pickle.dump(WMCA, f)

# Electricity consumption

In [3]:
# Electricity consumption data
elec_consump_df = pd.read_excel('../data/raw/LSOA_domestic_elec_2010-20.xlsx', sheet_name="2020", header=4)
elec_consump_df.columns = [
        'la_code', 'la', 'msoa_code', 'msoa', 'lsoa_code', 'lsoa', 'num_meter', 'total_consumption', 'mean_counsumption', 'median_consumption'
        ]
# Filter for local authorities in WMCA
elec_consump_df = elec_consump_df[elec_consump_df['la_code'].isin(WMCA_code)]

In [4]:
# Post code to LSOA to MSOA converting data
postcode_df = pd.read_csv("../data/raw/PCD_OA_LSOA_MSOA_LAD_AUG19_UK_LU.csv", low_memory=False)

with open('../data/raw/WMCA_council_code.pkl', 'rb') as f:
    WMCA_code = pickle.load(f)

# Filter for local authorities in WMCA
postcode_df = postcode_df[postcode_df['ladcd'].isin(WMCA_code)]

In [5]:
# Merge data to get postcodes associated with each LSOA code
postcode_elec_consump_df = pd.merge(postcode_df, elec_consump_df, left_on="lsoa11cd", right_on="lsoa_code", how="left")
postcode_elec_consump_df = postcode_elec_consump_df[['pcds', 'la', 'la_code', 'msoa_code', 'msoa', 'lsoa_code', 'lsoa', 'num_meter', 'total_consumption', 'mean_counsumption',
       'median_consumption']]

# EPC Rating Data

In [6]:
# Make private
import os
from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

AUTH_TOKEN = 'TWljaGFlbC5jb3VnaGxhbkB3YXJ3aWNrLmFjLnVrOjZlZWQwZTQ0OGFiMzljYzYxY2MwN2M4YmU0Y2VhYmQ5MTI2OGViZjI='

In [7]:
def get_epc_data(postcode, num_rows=5000):
    """
    Pull data from Domestic Energy Performance Certificates API.

    Input:
    postcode(str): (1) Postcode 
    num_rows(int): Number of rows to pull. Max 5000 allowed at one time

    Output:
    (str): Pulled data from API

    """
    headers = {
        'Authorization': f'Basic {AUTH_TOKEN}',
        'Accept': 'application/json'
    }
    params = {
        'postcode': postcode,
        'size': num_rows
    }
    url = f'https://epc.opendatacommunities.org/api/v1/domestic/search'
    res = requests.get(url, headers=headers, params=params)
    return res.text

In [8]:
# Pull WMCA postcode data and save as CSV
result = list()

for code in tqdm(postcode_elec_consump_df.pcds.unique()):
    requested_data = get_epc_data(code)
    if len(requested_data)!=0:
        result.extend(json.loads(requested_data)['rows'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.99it/s]


In [22]:
EPC_data = pd.DataFrame(result)

EPC_data['uprn'] = pd.to_numeric(EPC_data['uprn'],errors='coerce') # needs to be float for joining

# Merge EPC and electricity consumption data on postcode
EPC_postcode_elec_consump = pd.merge(EPC_data, postcode_elec_consump_df, left_on="postcode", right_on="pcds", how="left")
EPC_postcode_elec_consump.drop(columns=["pcds", "address1", "address2", "address3", 'uprn-source'], inplace=True)

0       10023504340
1       10023504345
2       10023504376
3       10023504370
4       10023504372
           ...     
1288    10023293700
1289    10023293854
1290    10023293863
1291    10023293751
1292    10023293684
Name: uprn, Length: 1293, dtype: object
0       1.002350e+10
1       1.002350e+10
2       1.002350e+10
3       1.002350e+10
4       1.002350e+10
            ...     
1288    1.002329e+10
1289    1.002329e+10
1290    1.002329e+10
1291    1.002329e+10
1292    1.002329e+10
Name: uprn, Length: 1293, dtype: float64


In [14]:
# Export postcodes
with open('../data/processed/WMCA_postcodes.pkl', 'wb') as fp:
    pickle.dump(EPC_postcode_elec_consump.postcode.unique(), fp)

# Fuel Poverty

In [15]:
fuel_poverty_df = pd.read_excel("../data/raw/sub-regional-fuel-poverty-2022-tables.xlsx", sheet_name="Table 3", header=2)
fuel_poverty_df.drop(columns=["LSOA Name", "LA Code", "LA Name", "Region", "Unnamed: 8"], inplace=True)
fuel_poverty_df.columns = ["lsoa_code", "num_households", "num_households_fuel_poverty", "prop_households_fuel_poor"]

In [16]:
EPC_postcode_elec_consump_fuel_poverty = pd.merge(EPC_postcode_elec_consump, fuel_poverty_df, on="lsoa_code", how="left")

# UPRN

In [17]:
uprn_df = pd.read_csv("../data/raw/osopenuprn_202205.csv")
uprn_df = uprn_df[['UPRN', 'LATITUDE', 'LONGITUDE']]

                  UPRN  X_COORDINATE  Y_COORDINATE   LATITUDE  LONGITUDE
0                    1     358260.66     172796.50  51.452601  -2.602075
1                   26     352967.00     181077.00  51.526633  -2.679361
2                   27     352967.00     181077.00  51.526633  -2.679361
3                   30     354800.00     180469.00  51.521317  -2.652862
4                   31     354796.00     180460.00  51.521236  -2.652918
...                ...           ...           ...        ...        ...
39975443  906700544182     260824.93     665005.14  55.858079  -4.225079
39975444  906700544183     252139.38     670727.41  55.906863  -4.366839
39975445  906700544184     255864.00     667585.17  55.879783  -4.305638
39975446  906700544185     251620.54     670994.73  55.909102  -4.375277
39975447  906700601612     257561.00     667814.00  55.882343  -4.278656

[39975448 rows x 5 columns]
                  UPRN   LATITUDE  LONGITUDE
0                    1  51.452601  -2.602075
1    

In [18]:
# Match column type with EPC UPRN so that we can merge
uprn_df.UPRN = uprn_df.UPRN.astype('object')

UPRN          object
LATITUDE     float64
LONGITUDE    float64
dtype: object


In [27]:
EPC_postcode_elec_consump_fuel_poverty_uprn = pd.merge(old_df, uprn_df, left_on="uprn", right_on="UPRN", how="left")
# EPC_postcode_elec_consump_fuel_poverty_uprn.drop(columns=["UPRN", 'la', 'la_code'], inplace=True)
EPC_postcode_elec_consump_fuel_poverty_uprn.drop(columns=["UPRN"], inplace=True)

0         -1.900206
1         -1.900206
2         -1.900206
3         -1.900206
4         -1.900206
             ...   
1012645   -2.152162
1012646   -2.152089
1012647   -2.151938
1012648   -2.151911
1012649   -2.151675
Name: LONGITUDE, Length: 1012650, dtype: float64


In [28]:
EPC_postcode_elec_consump_fuel_poverty_uprn.to_csv(f"../data/processed/data_{len(EPC_postcode_elec_consump_fuel_poverty_uprn)}.csv", index=False)